In [23]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import pandas as pd
import re
from tqdm import tqdm
import ssl
import sys
import warnings
warnings.filterwarnings('ignore')
import os
import numpy as np
import bs4.element
import time

In [24]:
# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
def makeUrl(search, page, maxpage_t, sort, s_date, e_date, s_from, e_to):
    urls = []
    if page == maxpage_t:       # 최대페이지 1일 경우
        url = "https://search.naver.com/search.naver?where=news&query=" + search + "&sort="+sort+"&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
        urls.append(url)
    else:
        while page <= maxpage_t :
            url = "https://search.naver.com/search.naver?where=news&query=" + search + "&sort="+sort+"&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
            urls.append(url)
            page += 10
    return urls    

# html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)
def news_attrs_crawler(articles,attrs):
    attrs_content=[]
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content

# ConnectionError방지
headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}

#html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    #html 불러오기
    original_html = requests.get(url, headers=headers, verify=False)

    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver,'href')
    return url

In [25]:
def crawling_all(search, page, maxpage_t, sort, s_date, e_date, s_from, e_to) :
    # naver url 생성
    urls = makeUrl(search, page, maxpage_t, sort, s_date, e_date, s_from, e_to)

    #뉴스 크롤러 실행
    news_titles = []
    news_url =[]
    news_contents =[]
    news_dates = []

    while page <= maxpage_t :
        url_origin = "https://search.naver.com/search.naver?where=news&query=" + search + "&sort="+sort+"&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
        url = articles_crawler(url_origin)
        news_url .append(url)
        page += 10

    #제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
    def makeList(newlist, content):
        for i in content:
            for j in i:
                newlist.append(j)
        return newlist

    #제목, 링크, 내용 담을 리스트 생성
    news_url_1 = []

    #1차원 리스트로 만들기(내용 제외)
    makeList(news_url_1,news_url)

    #NAVER 뉴스만 남기기
    final_urls = []
    for i in range(len(news_url_1)):
        if "news.naver.com" in news_url_1[i]:
            final_urls.append(news_url_1[i])
        else:
            pass
    
    for i in final_urls:
        #각 기사 html get하기
        headers = {'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}
        news = requests.get(i, headers=headers, verify=False)
        news_html = BeautifulSoup(news.text,"html.parser")

        # 뉴스 제목 가져오기
        title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
        if title == None:
            title = news_html.select_one("#content > div.end_ct > div > h2")
        
        # 뉴스 본문 가져오기
        content = news_html.select("div#dic_area")
        if content == []:
            content = news_html.select("#articeBody")
            
        # 기사 텍스트만 가져오기
        # list합치기
        content = ''.join(str(content))

        # html태그제거 및 텍스트 다듬기
        pattern1 = '<[^>]*>'
        title = re.sub(pattern=pattern1, repl='', string=str(title))
        content = re.sub(pattern=pattern1, repl='', string=content)
        pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
        content = content.replace(pattern2, '')

        news_titles.append(title)
        news_contents.append(content)

        # html_date = news_html.select_one("div#ct > div.media_end_head > div.media_end_head_info > div span")
        # news_date = html_date.attrs['data-date-time']
        # news_dates.append(news_date)

        try:
            html_date = news_html.select_one("div#ct > div.media_end_head > div.media_end_head_info > div span")
            news_date = html_date.attrs['data-date-time']
        except AttributeError:
            news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
            news_date = re.sub(pattern=pattern1,repl='',string=str(news_date))
        # 날짜 가져오기
        news_dates.append(news_date)

    a = pd.DataFrame({'date':news_dates,'title':news_titles,'link':final_urls,'content':news_contents})
    return a

In [26]:
# kospi = pd.read_csv('D:\\big16\\final project\\DATA\\KOSPI200.csv', encoding='euc-kr')
# kospi200 = kospi['종목명']
# kospi200

In [27]:
day = []
i = 7
while i<13:
    day.append('2021.{}.01'.format(str(i).zfill(2)))
    i += 1

print(day)

['2021.07.01', '2021.08.01', '2021.09.01', '2021.10.01', '2021.11.01', '2021.12.01']


In [28]:
i = 1
while i<13:
    day.append('2022.{}.01'.format(str(i).zfill(2)))
    i += 1

print(day)

['2021.07.01', '2021.08.01', '2021.09.01', '2021.10.01', '2021.11.01', '2021.12.01', '2022.01.01', '2022.02.01', '2022.03.01', '2022.04.01', '2022.05.01', '2022.06.01', '2022.07.01', '2022.08.01', '2022.09.01', '2022.10.01', '2022.11.01', '2022.12.01']


In [29]:
i = 1
while i<7:
    day.append('2023.{}.01'.format(str(i).zfill(2)))
    i += 1

print(day)

['2021.07.01', '2021.08.01', '2021.09.01', '2021.10.01', '2021.11.01', '2021.12.01', '2022.01.01', '2022.02.01', '2022.03.01', '2022.04.01', '2022.05.01', '2022.06.01', '2022.07.01', '2022.08.01', '2022.09.01', '2022.10.01', '2022.11.01', '2022.12.01', '2023.01.01', '2023.02.01', '2023.03.01', '2023.04.01', '2023.05.01', '2023.06.01']


In [30]:
#데이터 프레임 만들기
news_df = news_df = pd.DataFrame(
    data=None
    , index=None
    , columns=['date','title','link','content']
)

# 최대 페이지 수
maxpage = 401
#검색어 입력
search = '미국'
# 검색 방식
sort = '0'
# 시작날짜
s_date = '2021.06.28'
# 끝나는 날짜
e_date = '2021.06.30'

# 날짜 바꾸기
s_from = s_date.replace(".","")
e_to = e_date.replace(".","")

page = 1  
maxpage_t =(int(maxpage)-1)*10+1   # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지

a = crawling_all(search,page,maxpage_t, sort, s_date, e_date, s_from, e_to)
news_df = pd.concat([news_df, a], ignore_index=True, keys=['date','title','link','content'])
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)

now = datetime.now()
news_df.to_csv('D:\\big16\\final project\\DATA\\NEWS_DATA\\{}\\{}_date{}_{}.csv'.format(search, search,s_from, e_to),encoding='utf-8-sig',index=False)

In [31]:
#####뉴스크롤링 시작#####
for i in tqdm(day) :
    #데이터 프레임 만들기
    news_df = news_df = pd.DataFrame(
        data=None
        , index=None
        , columns=['date','title','link','content']
    )

    # 최대 페이지 수
    maxpage = 401
    #검색어 입력
    search = '미국'
    # 검색 방식
    sort = str(0)
    # 시작날짜
    s_date = i
    # 끝나는 날짜
    e_1 = i[-2:]
    e_2 = int(e_1)+30
    e_3 = i[:-2]+str(e_2)
    e_date = e_3

    # 날짜 바꾸기
    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")

    page = 1  
    maxpage_t =(int(maxpage)-1)*10+1   # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지

    a = crawling_all(search,page,maxpage_t, sort, s_date, e_date, s_from, e_to)
    news_df = pd.concat([news_df, a], ignore_index=True, keys=['date','title','link','content'])
    news_df = news_df.drop_duplicates(keep='first',ignore_index=True)

    now = datetime.now()
    news_df.to_csv('D:\\big16\\final project\\DATA\\NEWS_DATA\\{}\\{}_date{}_{}.csv'.format(search, search,s_from, e_to),encoding='utf-8-sig',index=False)

100%|██████████| 24/24 [36:51<00:00, 92.13s/it]  


In [32]:
#데이터 프레임 만들기
news_df = news_df = pd.DataFrame(
    data=None
    , index=None
    , columns=['date','title','link','content']
)

# 최대 페이지 수
maxpage = 401
#검색어 입력
search = '미국'
# 검색 방식
sort = '0'
# 시작날짜
s_date = '2023.07.01'
# 끝나는 날짜
e_date = '2023.07.11'

# 날짜 바꾸기
s_from = s_date.replace(".","")
e_to = e_date.replace(".","")

page = 1  
maxpage_t =(int(maxpage)-1)*10+1   # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지

a = crawling_all(search,page,maxpage_t, sort, s_date, e_date, s_from, e_to)
news_df = pd.concat([news_df, a], ignore_index=True, keys=['date','title','link','content'])
news_df = news_df.drop_duplicates(keep='first',ignore_index=True)

now = datetime.now()
news_df.to_csv('D:\\big16\\final project\\DATA\\NEWS_DATA\\{}\\{}_date{}_{}.csv'.format(search, search,s_from, e_to),encoding='utf-8-sig',index=False)

In [33]:
folders = os.listdir('D:\\big16\\final project\\DATA\\NEWS_DATA\\{}'.format(search))

In [34]:
df_all = pd.DataFrame()
for i in range(0,len(folders)):
    if folders[i].split('.')[1] == 'csv':
        file = 'D:\\big16\\final project\\DATA\\NEWS_DATA\\{}\\'.format(search)+folders[i]
        df= pd.read_csv(file,encoding='utf-8') 
        df_all = pd.concat([df_all, df])

In [35]:
df_all.head()

,date,title,linkcontent,link,content
0,2021-06-30 16:00:02,실수로 입금된 56조원…나흘간 세계 25위 부자된 미국 부부,NaN,https://n.news.naver.com/mnews/article/001/001...,"[\n은행 ""기술 오류로 고객 계좌 영향"" 즉각 신고했지만 나흘 뒤에야 회수""계좌에..."
1,2021-06-30 22:34:04,"""소련은 해방군, 미국은 점령군""...김원웅 광복회장, 고교생 대상 강연 논란",NaN,https://n.news.naver.com/mnews/article/029/000...,[\n\n\n\n\n광복회 관련 보도에 대한 입장 발표하는 김원웅 광복회장. [연합...
2,2021-06-30 14:59:03,미국 집값 4월 14.6% 폭등…왜 이렇게 오르나,NaN,https://n.news.naver.com/mnews/article/032/000...,[\n\t\t\t[경향신문] 미국에서도 주택시장 과열이 심각해지고 있다. 올 4월 ...
3,2021-06-30 22:13:01,"""델타 델타플러스 무섭게 퍼진다""…미국 러시아 한국 전세계가 초비상",NaN,https://n.news.naver.com/mnews/article/009/000...,"[\n\n\n\n\n[사진 제공 = 연합뉴스] 코로나19 바이러스에서 파생한 델타,..."
4,2021-06-30 07:35:18,"""너무 이뻐 눈을 못떼겠네"" 트렌스젠더 여성, 미국 미인대회 우승",NaN,https://n.news.naver.com/mnews/article/014/000...,[\n\n\n\n\n카탈루나 엔리케스. 카탈루나 엔리케스 인스타그램 캡쳐 미국 네바...


In [36]:
df_all.drop('link', axis=1, inplace=True)
# df_all.drop('Unnamed: 0', axis=1, inplace=True)
df_all.drop('linkcontent', axis=1, inplace=True)

In [37]:
import re

def clean_text(d):
  text = re.sub(r'\([^)]*\)', '', d)
  text = re.sub(r'\[[^]]*\]', '', text)
  text = re.sub(r'\<[^>]*\>', '', text)
  pattern = r'[^가-힣0-9a-zA-Z\s]'
  text = re.sub(pattern, ' ', text)
  text = re.sub(r'사진', ' ', text)
  text = re.sub(r'.*뉴스', ' ', text)
  text = re.sub("\n", ' ', text)
  text = re.sub("  +", " ", text)
  return text

def text_clean(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern, '', text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern, '', text)
    pattern = '[a-zA-Z0-9]'    # 숫자와 알파벳 제거
    text = re.sub(pattern, '', text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern, '', text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern, '', text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern, '', text)
    pattern = '[\n]'            # \n제거
    text = re.sub(pattern, '', text)
    pattern = '[\t]'            # \n제거
    text = re.sub(pattern, '', text)
    pattern = '[\']'           
    text = re.sub(pattern, '', text)
    pattern = '[\"]'            
    text = re.sub(pattern, '', text)
    return text  

In [38]:
df_all['content'] = df_all.content.apply(text_clean)
df_all['content'] = df_all.content.apply(clean_text)
df_all['title'] = df_all.content.apply(text_clean)
df_all['title'] = df_all.content.apply(clean_text)
df_all

,date,title,content
0,2021-06-30 16:00:02,등에 따르면 루이지애나주 배턴루지에서 부동산 중개회사를 운영하는 대런 제임스는 지...,등에 따르면 루이지애나주 배턴루지에서 부동산 중개회사를 운영하는 대런 제임스는 지...
1,2021-06-30 22:34:04,김원웅 광복회장이 고등학생을 대상으로 한 영상 강연에서 광복 이후 북한에 진입한 ...,김원웅 광복회장이 고등학생을 대상으로 한 영상 강연에서 광복 이후 북한에 진입한 ...
2,2021-06-30 14:59:03,경향신문 미국에서도 주택시장 과열이 심각해지고 있다 올 월 미국의 주택가격이 년 만...,경향신문 미국에서도 주택시장 과열이 심각해지고 있다 올 월 미국의 주택가격이 년 만...
3,2021-06-30 22:13:01,코로나 바이러스에서 파생한 델타 델타 플러스 변이 확산으로 전세계가 또다시 고비를...,코로나 바이러스에서 파생한 델타 델타 플러스 변이 확산으로 전세계가 또다시 고비를...
4,2021-06-30 07:35:18,위크 등에 따르면 성전환 여성 모델인 카탈루나 엔리케스는 미스 네바다주 대회에서 ...,위크 등에 따르면 성전환 여성 모델인 카탈루나 엔리케스는 미스 네바다주 대회에서 ...
...,...,...,...
11,2023-07-10 10:57:03,중국 제외한 전기차 판매량 전년비 증가테슬라 만대 인도하며 위 기록현대차기아 만대 ...,중국 제외한 전기차 판매량 전년비 증가테슬라 만대 인도하며 위 기록현대차기아 만대 ...
12,2023-07-11 16:15:00,무단 전재 및 재배포 금지남아프리카공화국이 세계에서 여행하기 가장 위험한 국가라는...,무단 전재 및 재배포 금지남아프리카공화국이 세계에서 여행하기 가장 위험한 국가라는...
13,2023-07-11 06:25:04,거리에서 마주치게 돼 정말 반갑고 지난 월 방한 시 환대에 매우 감사드린다고 인사...,거리에서 마주치게 돼 정말 반갑고 지난 월 방한 시 환대에 매우 감사드린다고 인사...
14,2023-07-11 10:59:51,반종빈 이재윤 기자 김여정 북한 노동당 부부장은 일 미국 공군 전략정찰기가 동해 ...,반종빈 이재윤 기자 김여정 북한 노동당 부부장은 일 미국 공군 전략정찰기가 동해 ...


In [39]:
df_all.dropna(axis=0, inplace=True)

In [40]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5601 entries, 0 to 15
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   date     5601 non-null   object
 1   title    5601 non-null   object
 2   content  5601 non-null   object
dtypes: object(3)
memory usage: 175.0+ KB


In [41]:
df_all.to_csv('D:\\big16\\final project\\DATA\\NEWS_DATA\\0. merge\\{}_merge.csv'.format(search))

In [42]:
import pymysql
conn = pymysql.connect(
    host='34.64.240.96'
    , user='root'
    , password='tndusWkd1.'
    , db='final_project'
    , charset='utf8'
)
cur = conn.cursor()

In [43]:
NewsDate = df_all['date'].to_list()
NewsTItle = df_all['title'].to_list()
NewsContent = df_all['content'].to_list()

In [44]:
for date,title,content in zip(NewsDate, NewsTItle, NewsContent):
    sql = "INSERT INTO past_news_usa VALUES ({}, {}, {})".format("\""+date+"\"", "\""+title+"\"", "\""+content+"\"")
    # print(sql)
    cur.execute(sql)
conn.commit()
conn.close()